In [ ]:
# ==========================================
# CELL 1: IMPORTS & SETUP
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import stats
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create charts directory
os.makedirs('../charts', exist_ok=True)

print("Libraries loaded successfully!")
print("\n" + "=" * 60)
print("CUSTOMER SEGMENTATION & A/B TESTING ANALYSIS")
print("=" * 60)

In [ ]:
# ==========================================
# CELL 2: LOAD DATA
# ==========================================

# Load the data
df = pd.read_csv('../data/processed/us_retail_ecommerce_data.csv')
df['Date'] = pd.to_datetime(df['Date'])

print("\nData loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

# Set analysis date (last date in dataset)
analysis_date = df['Date'].max()
print(f"Analysis date: {analysis_date}")

In [ ]:
# ==========================================
# CELL 3: RFM ANALYSIS (RECENCY, FREQUENCY, MONETARY)
# ==========================================

print("\n" + "=" * 60)
print("RFM ANALYSIS")
print("=" * 60)

# Calculate RFM metrics for each customer
rfm = df.groupby('Customer_ID').agg({
    'Date': lambda x: (analysis_date - x.max()).days,  # Recency
    'Order_ID': 'count',  # Frequency
    'Total_Amount': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['Customer_ID', 'Recency', 'Frequency', 'Monetary']

# Calculate RFM scores (1-5 scale)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Calculate overall RFM score
rfm['RFM_Score'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

print("\nRFM Score Distribution:")
print(rfm[['Recency', 'Frequency', 'Monetary', 'RFM_Score']].describe())

# Create customer segments based on RFM
def segment_customers(row):
    if row['RFM_Score'] >= 13:
        return 'Champions'
    elif row['RFM_Score'] >= 11:
        return 'Loyal'
    elif row['RFM_Score'] >= 9:
        return 'Potential'
    elif row['RFM_Score'] >= 7:
        return 'At Risk'
    else:
        return 'Lost'

rfm['Segment'] = rfm.apply(segment_customers, axis=1)

# Segment summary
segment_summary = rfm.groupby('Segment').agg({
    'Customer_ID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(2)

segment_summary.columns = ['Customer_Count', 'Avg_Recency_Days', 'Avg_Frequency', 'Avg_Monetary']
segment_summary['Pct_of_Customers'] = (segment_summary['Customer_Count'] / len(rfm) * 100).round(1)

print("\nCustomer Segments:")
print(segment_summary.sort_values('Avg_Monetary', ascending=False))

# Visualize RFM segments
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Customer count by segment
segment_order = segment_summary.sort_values('Avg_Monetary', ascending=False).index
colors_segment = {'Champions': '#2ECC71', 'Loyal': '#3498DB', 'Potential': '#F39C12', 
                   'At Risk': '#E67E22', 'Lost': '#E74C3C'}

axes[0, 0].bar(segment_order, 
               [segment_summary.loc[s, 'Customer_Count'] for s in segment_order],
               color=[colors_segment[s] for s in segment_order], alpha=0.8)
axes[0, 0].set_title('Customer Count by Segment', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Number of Customers', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='y')
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Average monetary value by segment
axes[0, 1].barh(segment_order,
                [segment_summary.loc[s, 'Avg_Monetary'] for s in segment_order],
                color=[colors_segment[s] for s in segment_order], alpha=0.8)
axes[0, 1].set_title('Average Customer Value by Segment', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Avg Monetary Value ($)', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Plot 3: Recency vs Frequency scatter
for segment in rfm['Segment'].unique():
    segment_data = rfm[rfm['Segment'] == segment]
    axes[1, 0].scatter(segment_data['Recency'], segment_data['Frequency'],
                       label=segment, alpha=0.6, s=50, color=colors_segment[segment])

axes[1, 0].set_title('Customer Recency vs Frequency by Segment', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Recency (days since last purchase)', fontsize=11)
axes[1, 0].set_ylabel('Frequency (number of purchases)', fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Segment distribution pie
axes[1, 1].pie([segment_summary.loc[s, 'Customer_Count'] for s in segment_order],
               labels=segment_order,
               colors=[colors_segment[s] for s in segment_order],
               autopct='%1.1f%%',
               startangle=90,
               textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1, 1].set_title('Customer Segment Distribution', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('../charts/rfm_segmentation.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nKEY INSIGHTS:")
print(f"   - {segment_summary.loc['Champions', 'Customer_Count']} Champions (top {segment_summary.loc['Champions', 'Pct_of_Customers']:.1f}%)")
print(f"   - Average Champion value: ${segment_summary.loc['Champions', 'Avg_Monetary']:,.2f}")
print(f"   - {segment_summary.loc['At Risk', 'Customer_Count']} customers At Risk")
print(f"   - {segment_summary.loc['Lost', 'Customer_Count']} Lost customers to win back")


In [ ]:
# ==========================================
# CELL 4: K-MEANS CLUSTERING
# ==========================================

print("\n" + "=" * 60)
print("K-MEANS CLUSTERING ANALYSIS")
print("=" * 60)

# Prepare data for clustering
cluster_data = rfm[['Recency', 'Frequency', 'Monetary']].copy()

# Standardize the features
scaler = StandardScaler()
cluster_data_scaled = scaler.fit_transform(cluster_data)

# Find optimal number of clusters using elbow method
inertias = []
K_range = range(2, 9)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(cluster_data_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_title('Elbow Method for Optimal K', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Number of Clusters (K)', fontsize=11)
axes[0].set_ylabel('Inertia', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Use K=4 for clustering
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(cluster_data_scaled)

# Analyze clusters
cluster_analysis = rfm.groupby('Cluster').agg({
    'Customer_ID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(2)

cluster_analysis.columns = ['Customer_Count', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary']
cluster_analysis['Pct_of_Base'] = (cluster_analysis['Customer_Count'] / len(rfm) * 100).round(1)

# Name clusters based on characteristics
def name_cluster(row):
    if row['Avg_Monetary'] > cluster_analysis['Avg_Monetary'].median() and row['Avg_Frequency'] > cluster_analysis['Avg_Frequency'].median():
        return 'High Value'
    elif row['Avg_Recency'] > cluster_analysis['Avg_Recency'].median():
        return 'Dormant'
    elif row['Avg_Frequency'] < cluster_analysis['Avg_Frequency'].median():
        return 'New/Occasional'
    else:
        return 'Regular'

cluster_analysis['Cluster_Name'] = cluster_analysis.apply(name_cluster, axis=1)

print("\nCluster Profiles:")
print(cluster_analysis.sort_values('Avg_Monetary', ascending=False))

# Visualize clusters
colors_cluster = ['#2ECC71', '#3498DB', '#F39C12', '#E74C3C']

axes[1].scatter(rfm['Frequency'], rfm['Monetary'], 
                c=rfm['Cluster'], cmap='viridis', 
                alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1].set_title('Customer Clusters (Frequency vs Monetary)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Frequency (purchases)', fontsize=11)
axes[1].set_ylabel('Monetary Value ($)', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Add cluster centers
centers = scaler.inverse_transform(kmeans.cluster_centers_)
axes[1].scatter(centers[:, 1], centers[:, 2], 
                marker='X', s=300, c='red', edgecolors='black', linewidth=2,
                label='Cluster Centers')
axes[1].legend()

plt.tight_layout()
plt.savefig('../charts/kmeans_clustering.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCLUSTER INSIGHTS:")
for idx, row in cluster_analysis.iterrows():
    print(f"\n   Cluster {idx} ({row['Cluster_Name']}):")
    print(f"      - {row['Customer_Count']} customers ({row['Pct_of_Base']:.1f}%)")
    print(f"      - Avg Value: ${row['Avg_Monetary']:,.2f}")
    print(f"      - Avg Frequency: {row['Avg_Frequency']:.1f} purchases")

In [ ]:
# ==========================================
# CELL 5: CUSTOMER LIFETIME VALUE (CLV) - FIXED
# ==========================================

print("\n" + "=" * 60)
print("CUSTOMER LIFETIME VALUE ANALYSIS")
print("=" * 60)

# Calculate average order value and purchase frequency
customer_metrics = df.groupby('Customer_ID').agg({
    'Total_Amount': ['sum', 'mean', 'count'],
    'Date': lambda x: (x.max() - x.min()).days
}).reset_index()

customer_metrics.columns = ['Customer_ID', 'Total_Spent', 'Avg_Order_Value', 'Purchase_Count', 'Customer_Lifespan_Days']

# FIX: Handle single-purchase customers (lifespan = 0)
# Set minimum lifespan to 30 days for calculation purposes
customer_metrics['Customer_Lifespan_Days'] = customer_metrics['Customer_Lifespan_Days'].replace(0, 30)

# Calculate purchase frequency (purchases per year)
customer_metrics['Purchase_Frequency'] = customer_metrics['Purchase_Count'] / (customer_metrics['Customer_Lifespan_Days'] / 365)
customer_metrics['Purchase_Frequency'] = customer_metrics['Purchase_Frequency'].clip(upper=50)  # Cap at 50 to avoid outliers

# Simple CLV calculation: AOV * Purchase Frequency * Average Customer Lifespan
# Assuming 2-year customer lifespan for calculation
assumed_lifespan_years = 2
customer_metrics['CLV'] = (customer_metrics['Avg_Order_Value'] * 
                           customer_metrics['Purchase_Frequency'] * 
                           assumed_lifespan_years)

# Merge with RFM segments
customer_metrics = customer_metrics.merge(rfm[['Customer_ID', 'Segment']], on='Customer_ID')

# CLV by segment
clv_by_segment = customer_metrics.groupby('Segment').agg({
    'CLV': 'mean',
    'Avg_Order_Value': 'mean',
    'Purchase_Frequency': 'mean',
    'Customer_ID': 'count'
}).round(2)

clv_by_segment.columns = ['Avg_CLV', 'Avg_Order_Value', 'Avg_Purchase_Freq', 'Customer_Count']
clv_by_segment = clv_by_segment.sort_values('Avg_CLV', ascending=False)

print("\nCustomer Lifetime Value by Segment:")
print(clv_by_segment)

# Visualize CLV
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: CLV by segment
axes[0, 0].barh(clv_by_segment.index, clv_by_segment['Avg_CLV'],
                color=[colors_segment[s] for s in clv_by_segment.index], alpha=0.8)
axes[0, 0].set_title('Average Customer Lifetime Value by Segment', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('CLV ($)', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (segment, clv) in enumerate(zip(clv_by_segment.index, clv_by_segment['Avg_CLV'])):
    axes[0, 0].text(clv + 5, i, f'${clv:,.0f}', va='center', fontsize=10, fontweight='bold')

# Plot 2: CLV distribution
axes[0, 1].hist(customer_metrics['CLV'], bins=50, color='#3498DB', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(customer_metrics['CLV'].median(), color='red', linestyle='--', 
                    linewidth=2, label=f'Median: ${customer_metrics["CLV"].median():.0f}')
axes[0, 1].set_title('Customer Lifetime Value Distribution', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('CLV ($)', fontsize=11)
axes[0, 1].set_ylabel('Number of Customers', fontsize=11)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: AOV vs Purchase Frequency by segment
for segment in customer_metrics['Segment'].unique():
    segment_data = customer_metrics[customer_metrics['Segment'] == segment]
    axes[1, 0].scatter(segment_data['Avg_Order_Value'], segment_data['Purchase_Frequency'],
                       label=segment, alpha=0.6, s=50, color=colors_segment[segment])

axes[1, 0].set_title('Average Order Value vs Purchase Frequency', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Average Order Value ($)', fontsize=11)
axes[1, 0].set_ylabel('Purchase Frequency (per year)', fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Total CLV by segment
total_clv_by_segment = customer_metrics.groupby('Segment')['CLV'].sum().sort_values(ascending=False)
axes[1, 1].bar(total_clv_by_segment.index, total_clv_by_segment.values,
               color=[colors_segment[s] for s in total_clv_by_segment.index], alpha=0.8)
axes[1, 1].set_title('Total CLV Pool by Segment', fontsize=13, fontweight='bold')
axes[1, 1].set_ylabel('Total CLV ($)', fontsize=11)
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../charts/customer_lifetime_value.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCLV INSIGHTS:")
print(f"   - Average CLV across all customers: ${customer_metrics['CLV'].mean():,.2f}")
print(f"   - Champions have {(clv_by_segment.loc['Champions', 'Avg_CLV'] / customer_metrics['CLV'].mean()):.1f}x higher CLV")
print(f"   - Total customer value in database: ${customer_metrics['CLV'].sum():,.2f}")


In [ ]:
# ==========================================
# CELL 6: A/B TEST SIMULATION - DISCOUNT STRATEGY
# ==========================================

print("\n" + "=" * 60)
print("A/B TEST SIMULATION: DISCOUNT STRATEGY")
print("=" * 60)

# Simulate A/B test: 10% discount vs 15% discount
# We'll use existing data and simulate what would happen with different discount levels

# Get baseline metrics
baseline_data = df[df['Discount_Amount'] > 0].copy()
baseline_avg_discount_rate = (baseline_data['Discount_Amount'] / 
                               (baseline_data['Total_Amount'] + baseline_data['Discount_Amount'])).mean()

print(f"\nBaseline average discount rate: {baseline_avg_discount_rate*100:.1f}%")

# Simulate two scenarios
np.random.seed(42)

# Split customers into two groups
unique_customers = df['Customer_ID'].unique()
np.random.shuffle(unique_customers)
split_point = len(unique_customers) // 2

group_a_customers = unique_customers[:split_point]  # 10% discount group
group_b_customers = unique_customers[split_point:]  # 15% discount group

# Create test groups
df_test = df.copy()
df_test['Test_Group'] = df_test['Customer_ID'].apply(
    lambda x: 'A (10% Discount)' if x in group_a_customers else 'B (15% Discount)'
)

# Simulate behavior changes based on discount
# Group A: 10% discount - moderate conversion boost
df_test.loc[df_test['Test_Group'] == 'A (10% Discount)', 'Simulated_Conversion_Boost'] = 1.08

# Group B: 15% discount - higher conversion boost but lower margin
df_test.loc[df_test['Test_Group'] == 'B (15% Discount)', 'Simulated_Conversion_Boost'] = 1.15

# Calculate simulated metrics
df_test['Simulated_Revenue'] = df_test['Total_Amount'] * df_test['Simulated_Conversion_Boost']

# Adjust for discount
df_test.loc[df_test['Test_Group'] == 'A (10% Discount)', 'Simulated_Discount'] = df_test['Total_Amount'] * 0.10
df_test.loc[df_test['Test_Group'] == 'B (15% Discount)', 'Simulated_Discount'] = df_test['Total_Amount'] * 0.15

df_test['Simulated_Net_Revenue'] = df_test['Simulated_Revenue'] - df_test['Simulated_Discount']

# Analyze results
ab_results = df_test.groupby('Test_Group').agg({
    'Customer_ID': 'nunique',
    'Order_ID': 'count',
    'Simulated_Revenue': 'sum',
    'Simulated_Discount': 'sum',
    'Simulated_Net_Revenue': 'sum'
}).round(2)

ab_results.columns = ['Unique_Customers', 'Total_Orders', 'Gross_Revenue', 'Total_Discount', 'Net_Revenue']
ab_results['Avg_Order_Value'] = (ab_results['Gross_Revenue'] / ab_results['Total_Orders']).round(2)
ab_results['ROI'] = ((ab_results['Net_Revenue'] - ab_results['Total_Discount']) / ab_results['Total_Discount'] * 100).round(2)

print("\nA/B Test Results:")
print(ab_results)

# Statistical significance test
group_a_values = df_test[df_test['Test_Group'] == 'A (10% Discount)']['Simulated_Net_Revenue']
group_b_values = df_test[df_test['Test_Group'] == 'B (15% Discount)']['Simulated_Net_Revenue']

t_stat, p_value = stats.ttest_ind(group_a_values, group_b_values)

print(f"\nStatistical Test:")
print(f"   - t-statistic: {t_stat:.4f}")
print(f"   - p-value: {p_value:.4f}")
print(f"   - Result: {'Statistically significant' if p_value < 0.05 else 'Not significant'} (α=0.05)")

# Visualize A/B test results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Revenue comparison
metrics = ['Gross_Revenue', 'Total_Discount', 'Net_Revenue']
x = np.arange(len(metrics))
width = 0.35

group_a_vals = [ab_results.loc['A (10% Discount)', m] for m in metrics]
group_b_vals = [ab_results.loc['B (15% Discount)', m] for m in metrics]

axes[0, 0].bar(x - width/2, group_a_vals, width, label='Group A (10%)', color='#3498DB', alpha=0.8)
axes[0, 0].bar(x + width/2, group_b_vals, width, label='Group B (15%)', color='#E74C3C', alpha=0.8)
axes[0, 0].set_title('Revenue Metrics by Test Group', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Amount ($)', fontsize=11)
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(['Gross Revenue', 'Discount Cost', 'Net Revenue'])
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Plot 2: Average order value
axes[0, 1].bar(ab_results.index, ab_results['Avg_Order_Value'],
               color=['#3498DB', '#E74C3C'], alpha=0.8)
axes[0, 1].set_title('Average Order Value by Group', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('AOV ($)', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].tick_params(axis='x', rotation=15)

# Plot 3: Net revenue distribution
axes[1, 0].hist([group_a_values, group_b_values], bins=50, 
                label=['Group A (10%)', 'Group B (15%)'],
                color=['#3498DB', '#E74C3C'], alpha=0.6)
axes[1, 0].set_title('Net Revenue Distribution by Group', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Net Revenue ($)', fontsize=11)
axes[1, 0].set_ylabel('Frequency', fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Conversion lift and margin impact
lift_data = pd.DataFrame({
    'Metric': ['Conversion\nLift', 'Net Revenue\nLift', 'Margin\nImpact'],
    'Group A': [8, ((ab_results.loc['A (10% Discount)', 'Net_Revenue'] / 
                     ab_results.loc['B (15% Discount)', 'Net_Revenue'] - 1) * 100), -10],
    'Group B': [15, 0, -15]
})

x_pos = np.arange(len(lift_data))
axes[1, 1].bar(x_pos - 0.2, lift_data['Group A'], 0.4, label='Group A (10%)', 
               color='#3498DB', alpha=0.8)
axes[1, 1].bar(x_pos + 0.2, lift_data['Group B'], 0.4, label='Group B (15%)', 
               color='#E74C3C', alpha=0.8)
axes[1, 1].set_title('Performance Comparison', fontsize=13, fontweight='bold')
axes[1, 1].set_ylabel('Percentage (%)', fontsize=11)
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(lift_data['Metric'])
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../charts/ab_test_results.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate incremental revenue
group_a_net = ab_results.loc['A (10% Discount)', 'Net_Revenue']
group_b_net = ab_results.loc['B (15% Discount)', 'Net_Revenue']
incremental_revenue = group_b_net - group_a_net
incremental_pct = (incremental_revenue / group_a_net) * 100

print("\nTEST INSIGHTS:")
print(f"   - Group B (15% discount) generates {incremental_pct:+.1f}% {'more' if incremental_revenue > 0 else 'less'} net revenue")
print(f"   - Incremental revenue: ${incremental_revenue:,.2f}")
print(f"   - Winner: {'Group B' if group_b_net > group_a_net else 'Group A'}")

print("\nRECOMMENDATION:")
if group_b_net > group_a_net and p_value < 0.05:
    print(f"   ✓ Implement 15% discount strategy")
    print(f"   ✓ Expected annual revenue increase: ${incremental_revenue * 2:,.2f}")
elif group_a_net > group_b_net and p_value < 0.05:
    print(f"   ✓ Implement 10% discount strategy")
    print(f"   ✓ Better margin preservation while maintaining conversion")
else:
    print(f"   ⚠ Results not statistically significant - continue testing")



In [ ]:
# ==========================================
# CELL 7: TARGETED CAMPAIGN RECOMMENDATIONS
# ==========================================

print("\n" + "=" * 60)
print("TARGETED CAMPAIGN RECOMMENDATIONS")
print("=" * 60)

# Merge all segmentation data
customer_complete = rfm.merge(customer_metrics[['Customer_ID', 'CLV', 'Avg_Order_Value']], 
                               on='Customer_ID')

# Define campaign strategies for each segment
campaign_recommendations = {
    'Champions': {
        'Strategy': 'VIP Loyalty Program',
        'Discount': '5%',
        'Channel': 'Email + SMS',
        'Expected_Response': '25%',
        'Expected_ROI': '8.5x'
    },
    'Loyal': {
        'Strategy': 'Exclusive Early Access',
        'Discount': '10%',
        'Channel': 'Email',
        'Expected_Response': '18%',
        'Expected_ROI': '6.2x'
    },
    'Potential': {
        'Strategy': 'Cross-sell Campaign',
        'Discount': '12%',
        'Channel': 'Email + Retargeting',
        'Expected_Response': '12%',
        'Expected_ROI': '4.8x'
    },
    'At Risk': {
        'Strategy': 'Win-back Campaign',
        'Discount': '20%',
        'Channel': 'Email + Direct Mail',
        'Expected_Response': '8%',
        'Expected_ROI': '3.2x'
    },
    'Lost': {
        'Strategy': 'Reactivation Offer',
        'Discount': '25%',
        'Channel': 'Email + Paid Social',
        'Expected_Response': '5%',
        'Expected_ROI': '2.1x'
    }
}

# Create recommendations dataframe
recommendations_df = pd.DataFrame(campaign_recommendations).T
recommendations_df = recommendations_df.merge(
    segment_summary[['Customer_Count', 'Avg_Monetary']], 
    left_index=True, right_index=True
)

print("\nCampaign Recommendations by Segment:")
print(recommendations_df)

# Calculate expected campaign results
recommendations_df['Expected_Revenue'] = (
    recommendations_df['Customer_Count'] * 
    recommendations_df['Expected_Response'].str.rstrip('%').astype(float) / 100 * 
    recommendations_df['Avg_Monetary']
)

recommendations_df['Campaign_Cost'] = recommendations_df['Customer_Count'] * 5  # $5 per customer contact cost

recommendations_df['Expected_Profit'] = (
    recommendations_df['Expected_Revenue'] - 
    recommendations_df['Campaign_Cost']
)

print("\nExpected Campaign Performance:")
print(recommendations_df[['Customer_Count', 'Expected_Revenue', 'Campaign_Cost', 'Expected_Profit']].round(2))

# Visualize recommendations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Expected revenue by segment
axes[0, 0].barh(recommendations_df.index, recommendations_df['Expected_Revenue'],
                color=[colors_segment[s] for s in recommendations_df.index], alpha=0.8)
axes[0, 0].set_title('Expected Campaign Revenue by Segment', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Expected Revenue ($)', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Plot 2: ROI comparison
roi_values = recommendations_df['Expected_ROI'].str.rstrip('x').astype(float)
axes[0, 1].bar(recommendations_df.index, roi_values,
               color=[colors_segment[s] for s in recommendations_df.index], alpha=0.8)
axes[0, 1].axhline(y=5.0, color='green', linestyle='--', linewidth=2, 
                    alpha=0.5, label='Target ROI (5x)')
axes[0, 1].set_title('Expected Campaign ROI by Segment', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('ROI', fontsize=11)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Discount strategy
discount_values = recommendations_df['Discount'].str.rstrip('%').astype(float)
axes[1, 0].bar(recommendations_df.index, discount_values,
               color=[colors_segment[s] for s in recommendations_df.index], alpha=0.8)
axes[1, 0].set_title('Recommended Discount by Segment', fontsize=13, fontweight='bold')
axes[1, 0].set_ylabel('Discount (%)', fontsize=11)
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].tick_params(axis='x', rotation=45)

# Plot 4: Expected profit
axes[1, 1].barh(recommendations_df.index, recommendations_df['Expected_Profit'],
                color=[colors_segment[s] for s in recommendations_df.index], alpha=0.8)
axes[1, 1].set_title('Expected Campaign Profit by Segment', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Expected Profit ($)', fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../charts/campaign_recommendations.png', dpi=300, bbox_inches='tight')
plt.show()

total_expected_revenue = recommendations_df['Expected_Revenue'].sum()
total_expected_profit = recommendations_df['Expected_Profit'].sum()

print("\nTOTAL CAMPAIGN IMPACT:")
print(f"   - Total expected revenue: ${total_expected_revenue:,.2f}")
print(f"   - Total campaign cost: ${recommendations_df['Campaign_Cost'].sum():,.2f}")
print(f"   - Total expected profit: ${total_expected_profit:,.2f}")
print(f"   - Overall campaign ROI: {(total_expected_revenue / recommendations_df['Campaign_Cost'].sum()):.2f}x")


In [ ]:
# ==========================================
# CELL 8: EXECUTIVE SUMMARY - FIXED
# ==========================================

print("\n" + "=" * 60)
print("EXECUTIVE SUMMARY: CUSTOMER SEGMENTATION INSIGHTS")
print("=" * 60)

summary = f"""
CUSTOMER BASE OVERVIEW:
   - Total Customers Analyzed: {len(rfm):,}
   - Customer Segments Identified: 5 (RFM-based) + 4 (Clustering-based)
   - Average Customer Lifetime Value: ${customer_metrics['CLV'].mean():,.2f}
   - Total Customer Value in Database: ${customer_metrics['CLV'].sum():,.2f}

TOP SEGMENTS:
   - Champions: {segment_summary.loc['Champions', 'Customer_Count']:,} customers ({segment_summary.loc['Champions', 'Pct_of_Customers']:.1f}%)
     > Average Value: ${segment_summary.loc['Champions', 'Avg_Monetary']:,.2f}
     > Average CLV: ${clv_by_segment.loc['Champions', 'Avg_CLV']:,.2f}
     
   - At Risk: {segment_summary.loc['At Risk', 'Customer_Count']:,} customers ({segment_summary.loc['At Risk', 'Pct_of_Customers']:.1f}%)
     > Require immediate win-back campaign
     > Potential revenue at risk: ${segment_summary.loc['At Risk', 'Customer_Count'] * segment_summary.loc['At Risk', 'Avg_Monetary']:,.2f}

A/B TEST RESULTS:
   - Test: 10% Discount vs 15% Discount
   - Winner: {"15% Discount" if group_b_net > group_a_net else "10% Discount"}
   - Incremental Impact: {incremental_pct:+.1f}% net revenue change
   - Statistical Significance: {"Yes" if p_value < 0.05 else "No"} (p={p_value:.4f})

CAMPAIGN RECOMMENDATIONS:
   - Total Expected Revenue: ${total_expected_revenue:,.2f}
   - Total Campaign Investment: ${recommendations_df['Campaign_Cost'].sum():,.2f}
   - Expected Profit: ${total_expected_profit:,.2f}
   - Overall Campaign ROI: {(total_expected_revenue / recommendations_df['Campaign_Cost'].sum()):.2f}x

TOP 3 PRIORITY ACTIONS:
   1. Launch VIP program for {segment_summary.loc['Champions', 'Customer_Count']} Champions
      Expected return: {roi_values['Champions']:.1f}x ROI
      
   2. Deploy win-back campaign for {segment_summary.loc['At Risk', 'Customer_Count']} At-Risk customers
      Protect ${segment_summary.loc['At Risk', 'Customer_Count'] * segment_summary.loc['At Risk', 'Avg_Monetary']:,.2f} in potential revenue
      
   3. Implement {"15%" if group_b_net > group_a_net else "10%"} discount strategy based on A/B test results
      Expected annual impact: ${abs(incremental_revenue) * 2:,.2f}
"""

print(summary)

# Save summary to file with UTF-8 encoding
with open('../insights_summary_segmentation.txt', 'w', encoding='utf-8') as f:
    f.write("CUSTOMER SEGMENTATION & A/B TESTING ANALYSIS\n")
    f.write("=" * 60 + "\n\n")
    f.write(summary)
    f.write(f"\n\nAnalysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE!")
print("=" * 60)
print("\nAll visualizations saved to ../charts/")
print("\nNew files created:")
print("   - rfm_segmentation.png")
print("   - kmeans_clustering.png")
print("   - customer_lifetime_value.png")
print("   - ab_test_results.png")
print("   - campaign_recommendations.png")
print("\nSummary saved to: insights_summary_segmentation.txt")